In [2]:
import numpy as np
import nltk # type: ignore 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize # type: ignore
from nltk.corpus import stopwords # type: ignore
import string
import numpy as np
from preprocess import extract_features, build_freq, word_freq


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
file_path_positive = '../data/positive_tweets.txt'
file_path_negative = '../data/negative_tweets.txt'

pos_freqs = build_freq(file_path_positive)  # Build frequency dictionary
# print("The psoitive frequency")
# print(pos_freqs)
neg_freqs = build_freq(file_path_negative)
# print("The negative frquency")
# print(neg_freqs)
word_f = word_freq(pos_freqs, neg_freqs, "../data/sample_sentiment_text.txt")
#print("The frequency of the sentences ares ", word_f)
matrix = extract_features(word_f)
print(matrix)

[[ 1.  9.  5.]
 [ 1.  4.  7.]
 [ 1.  6.  1.]
 [ 1.  7.  0.]
 [ 1.  2.  7.]
 [ 1.  1.  2.]
 [ 1.  2.  1.]
 [ 1.  1.  0.]
 [ 1.  3.  3.]
 [ 1.  1.  1.]
 [ 1.  3.  3.]
 [ 1.  0.  0.]
 [ 1.  2.  2.]
 [ 1. 12.  0.]
 [ 1.  2.  5.]
 [ 1.  0.  7.]]


In [20]:
#I am building my classification model
#the logixtic model
import math
#initialize the parameters
b = 0.05
parameter = np.array([0.001, 0.01, 0.01])
y_hat = []
for row in matrix:
    feature = row
    z = np.dot(feature, parameter) + b
    g = 1/(1+(math.e)**(z))
    y_hat.append(g)
print(y_hat)
    


[0.45239463635418925, 0.45983671857787534, 0.46978685356447, 0.46978685356447, 0.46480828456469886, 0.4797610644281854, 0.4797610644281854, 0.4847547270119201, 0.4722784572508355, 0.48225745272227183, 0.4722784572508355, 0.4872527628438865, 0.47726568640604844, 0.4573538673545378, 0.46978685356447, 0.46978685356447]


In [28]:
import pandas as pd
df = pd.read_csv("../data/output.txt")
print(df)
m = len(df)
target = df.values
print(target)

    1
0   0
1   2
2   1
3   0
4   1
5   0
6   0
7   1
8   1
9   1
10  1
11  1
12  1
13  0
[[0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]]


In [31]:
#getting my cost and loss functions
import math
#compute the loss function
def loss_function():
    cost = 0
    for x in range(m):
        output = target[x,0]
        loss = (-1 *output * math.log(y_hat[x])) - (1-output) * math.log(1-y_hat[x])
        print("loss = ", loss)
        cost +=loss
    return cost
cost = loss_function()
cost =  cost/m
print("cost =", cost)

loss =  0.6022003907914684
loss =  0.9378838121253865
loss =  0.7554761901959626
loss =  0.634476190195963
loss =  0.7661302496689021
loss =  0.6534670814562904
loss =  0.6534670814562904
loss =  0.7241122334640739
loss =  0.7501865155471161
loss =  0.7292771732519026
loss =  0.7501865155471161
loss =  0.718972270330632
loss =  0.7396819485957433
loss =  0.6112978609090919
cost = 0.71620110810971


In [17]:

#then compute the gradient
rate = 0.0001
def gradient_descent():
    #gradient_functuion goes here